In [ ]:
from IPython.display import Image, display

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.config.system_parameters import TriageSettings

from react_agent.src.util.tools_fabric import ToolsFabric

In [ ]:
USE_MCP = False
DEBUG_MODE = True
USE_TRIAGE = False

In [ ]:
mapping_question = "Explain how 'Payment Terms' is mapped. Start with 'map_invoice1'."
extensibility_question = "As a Public Cloud customer in Spain, can I extend an existing eDocument customer invoice Process?"
peppol_support = "We as Syntax are currently implementing SAP DRC and are following the official SAP help. We are currently at the step “Integration of Peppol Exchange with SAP S/4HANA Cloud” and there at the creation of an S/4HANA Public Cloud communication user. A client certificate should be uploaded for the communication user to be created, as Basic Auth (via user + password) is not supported. The problem: We do not have this certificate. Where can I get this certificate? What format must it be in? We are stuck at this step because we are missing the certificate mentioned and it is also not clear to us what kind of certificate is meant here and where to get it."
france_us = "As a localization developer, I want to enhance the relevancy check in the France eDocument country class with the new source type so that settlement management documents become relevant for the eDocument process and eDocuments are created for the source documents. How can I Achieve this?"

QUERY = france_us

In [ ]:
if USE_TRIAGE:
    triage_agent = Triage()

    query_categorization = triage_agent.triage_user_message(user_message=QUERY)
    if DEBUG_MODE:
        print(f"query categorization: {query_categorization}")

In [ ]:
if USE_TRIAGE:
    tools = ToolsFabric.get_tools_for_category(
        use_mcp=USE_MCP,
        configuration=query_categorization["category"],
    )
else:
    tools = ToolsFabric.get_tools_for_category(
        use_mcp=USE_MCP,
        configuration=TriageSettings.Categories.ALL,
    )

In [ ]:
if USE_MCP:
    async with MultiServerMCPClient(tools) as client:
        agent = ReActAgent(tool_list=client.get_tools())
        if DEBUG_MODE:
            print(agent.get_system_prompt())
            display(Image(agent.get_agent_graph()))

        await agent.arun_agent_with_input(user_message=QUERY, debug=DEBUG_MODE)
else:
    agent = ReActAgent(tool_list=tools)
    if DEBUG_MODE:
        print(agent.get_system_prompt())
        display(Image(agent.get_agent_graph()))

    agent.run_agent_with_input(user_message=QUERY, debug=DEBUG_MODE)

In [ ]:
run_data = agent.get_execution_data()

run_data.pretty_print()

In [ ]:
print(run_data.final_output)